In [1]:
import julia

julia.install()

[ Info: Julia version info


Julia Version 1.5.4
Commit 69fcb5745b (2021-03-11 19:13 UTC)
Platform Info:
  OS: macOS (x86_64-apple-darwin18.7.0)
  uname: Darwin 20.5.0 Darwin Kernel Version 20.5.0: Sat May  8 05:10:33 PDT 2021; root:xnu-7195.121.3~9/RELEASE_X86_64 x86_64 i386
  CPU: Intel(R) Core(TM) i9-9980HK CPU @ 2.40GHz: 
                 speed         user         nice          sys         idle          irq
       #1-16  2400 MHz     496241 s          0 s     221932 s   19724852 s          0 s
       
  Memory: 64.0 GB (38813.4375 MB free)
  Uptime: 218968.0 sec
  Load Avg:  1.52294921875  1.84716796875  2.00537109375
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-9.0.1 (ORCJIT, skylake)
Environment:
  TERM = xterm-color
  PATH = /Users/kwat/.pyenv/versions/3.9.6/bin:/usr/local/Cellar/pyenv/2.0.3/libexec:/usr/local/Cellar/pyenv/2.0.3/plugins/python-build/bin:/Users/kwat/.pyenv/shims:/Users/kwat/.pyenv/bin:/usr/local/bin:/usr/bin:/bin:/usr/sbin:/sbin:/Library/TeX/texbin
  PYENV_HOOK_PATH = /Users/kwat/.py

[ Info: Julia executable: /Applications/Julia-1.5.app/Contents/Resources/julia/bin/julia
[ Info: Trying to import PyCall...
┌ Info: PyCall is already installed and compatible with Python executable.
│ 
│ PyCall:
│     python: /Users/kwat/.pyenv/versions/3.9.6/bin/python
│     libpython: /Users/kwat/.pyenv/versions/3.9.6/lib/libpython3.9.dylib
│ Python:
│     python: /Users/kwat/.pyenv/versions/3.9.6/bin/python
└     libpython: /Users/kwat/.pyenv/versions/3.9.6/lib/libpython3.9.dylib


In [2]:
from julia import Main

In [3]:
Main.eval(
    """
using DataFrames
using Pandas
using Plotly

using GCTGMT 
using FeatureSetEnrichment
"""
)

JuliaError: Exception 'ArgumentError: Package DataFrames not found in current path:
- Run `import Pkg; Pkg.add("DataFrames")` to install the DataFrames package.
' occurred while calling julia code:

using DataFrames
using Pandas
using Plotly

using GCTGMT 
using FeatureSetEnrichment


In [ ]:
import pandas as pd

In [ ]:
score_ = pd.read_csv(
    "/Users/kwat/code/jl/FeatureSetEnrichment.jl/notebook/data/gene_score.tsv",
    sep="\t",
    index_col=0,
    squeeze=True,
)

element_x_sample = pd.concat((score_, score_ * 10, score_ / 10), axis=1).reset_index()

element_x_sample.columns = (
    "Sample {}".format(i) for i in range(element_x_sample.shape[1])
)

set_to_element_ = Main.eval(
    """
read_gmt("/Users/kwat/code/jl/FeatureSetEnrichment.jl/notebook/data/c2.all.v7.1.symbols.gmt")
"""
)

set_element_ = set_to_element_["COLLER_MYC_TARGETS_UP"]

In [ ]:
def send(score_, set_element, set_to_element_, element_x_sample):

    if score_ is not None:

        Main.element_ = list(score_.index)

        Main.score_ = list(score_)

    if set_element_ is not None:

        Main.set_element_ = list(set_element_)

    if set_to_element_ is not None:

        Main.set_to_element_ = set_to_element_

        Main.eval(
            """
        set_to_element_ = convert(Dict{String, Vector{String}}, set_to_element_)
        """
        )

    if element_x_sample is not None:

        Main.element_x_sample = element_x_sample

        Main.eval(
            """
        element_x_sample = DataFrames.DataFrame(Pandas.DataFrame(element_x_sample))
        """
        )

In [ ]:
def score_1_sample_1_set(
    score_, set_element_, sort=True, plot=True, title_text="Title"
):

    send(score_, set_element_, None, None)

    Main.sort = sort

    Main.plot = plot

    Main.title_text = title_text

    result = Main.eval(
        """
    score_set(
        element_,
        score_,
        set_element_;
        sort = sort,
        plot = plot,
        title_text = title_text,
    )
    """
    )

    return result


score_1_sample_1_set(score_, set_element_)

In [ ]:
def score_1_sample_n_set(score_, set_to_element_, sort=True):

    send(score_, None, set_to_element_, None)

    Main.sort = sort

    set_to_retsult = Main.eval(
        """
    score_set(
        element_,
        score_,
        set_to_element_;
        sort = sort,
    )
    """
    )

    return set_to_retsult


score_1_sample_n_set(score_, set_to_element_)

In [ ]:
def score_n_sample_n_set(element_x_sample, set_to_element_, n_job=1):

    send(None, None, set_to_element_, element_x_sample)

    Main.n_job = n_job

    set_x_sample = Main.eval(
        """
    Pandas.DataFrame(score_set(
        element_x_sample,
        set_to_element_;
        n_job = n_job,
    ))
    """
    )

    return set_x_sample.set_index("Set")


score_n_sample_n_set(element_x_sample, set_to_element_)